Code for the forward pass across the q-network and for policies

Inspired by open ai baselines deepq

TODO Bayesian w/ dropout!

https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf

TODO - deep-rl-tensorflow networks (dueling, plus the structure in network.py (Network class)

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

def feed_forward(hiddens, observation, num_actions, scope, layer_norm=False):
    """
THIS IS JUST A SAMPLE - more developed version is in networks.ipynb
    
    """
    with tf.variable_scope(scope, reuse=False):
        out = observation
        
        for hidden in hiddens:
            out = layers.fully_connected(out, num_outputs=hidden, activation_fn=None)
            
            if layer_norm:
                out = layers.layer_norm(out, center=True, scale=True)
                
            layer = tf.nn.relu(out)

    return layers.fully_connected(layer, num_outputs=num_actions, activation_fn=None)
    
    

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
hiddens = (50, 50)
observation = tf.placeholder(shape=(None,2), dtype=tf.float32)
num_actions = 3
scope = 'online'

q_values = feed_forward(hiddens, observation, num_actions, scope, layer_norm=True)

In [3]:
def greedy_policy(q_values):
    return tf.argmax(q_values, axis=1)    
    
greedy = greedy_policy(q_values)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    q, g = sess.run([q_values, greedy], {observation: np.arange(6).reshape(3, -1)})
    print(q)
    print(g)    

[[ 0.35826737  0.54035956  0.2509272 ]
 [-0.15241034  0.7827507   0.1708551 ]
 [-0.20343663  0.8433749   0.13169017]]
[1 1 1]


In [4]:
def e_greedy_policy(q_values):
    epsilon = tf.placeholder(shape=(), dtype=tf.float32, name='epsilon')


In [5]:
bs = 3
#  requires an array as input
r = tf.random_uniform(tf.stack([bs]),
                      minval=0,
                      maxval=num_actions,
                      dtype=tf.int64)
with tf.Session() as sess:
    out = sess.run(r)

In [6]:
out

array([1, 1, 2])

In [7]:
def softmax_policy(q_values):
    """
    
    Basic structure of the policy from here
    https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf
    
    Calculation of entropy from here
    https://github.com/dennybritz/reinforcement-learning/issues/34
    
    Log of the probabilities are taken for speed, accuracy and stability 
    """
    temperature = tf.placeholder(shape=(), dtype=tf.float32, name='softmax_temperature')
    softmax = tf.nn.softmax(tf.divide(q_values, temperature), 
                            axis=1)
    log_probs = tf.log(softmax)
    
    entropy = -tf.reduce_sum(softmax * log_probs, 1, name='softmax_entropy')
    
    samples = tf.multinomial(log_probs, num_samples=1)
    
    return temperature, log_probs, entropy, samples

temperature, probs, entropy, samples = softmax_policy(q_values)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    t, p, e, s = sess.run([temperature, probs, entropy, samples], 
                          {observation: np.arange(6).reshape(3, -1), 
                           temperature: 0.5})
    
print(t, p, e, s)

0.5 [[-2.8370497  -0.14282192 -2.5970454 ]
 [-0.82877815 -1.5227816  -1.0633032 ]
 [-0.78820354 -1.9261627  -0.91720873]] [0.48352242 1.0611279  1.005566  ] [[1]
 [0]
 [2]]
